In [13]:
import pandas as pd
from pandas.plotting import table
import numpy as np  
import matplotlib.pyplot as plt
import openpyxl
from IPython.display import HTML
import pingouin as pg
import os
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [2]:
xl_usu = 'usu_individual_T125.xlsx'  
dfusu = pd.read_excel(xl_usu)
dfusu = dfusu.loc[:, ['CODUSU','NRO_HOGAR','COMPONENTE', 'ESTADO','V5_01_M','V5_02_M','V5_03_M','PONDERA']]

In [3]:
xl_hog = 'usu_hogar_T125.xlsx'  
dfhog = pd.read_excel(xl_hog)
dfhog = dfhog.loc[:, ['CODUSU','NRO_HOGAR','DECIFR','REGION','V5_01','V5_02','V5_03']]

In [4]:
# V5 = 1 si ALGUNA variable es 1, 2 solo si TODAS son 2
dfhog['V5'] = 2  # Inicializar con 2 (asumiendo que todas son 2)

# Si ALGUNA variable es 1, entonces V5 = 1
mask_alguna_1 = (dfhog['V5_01'] == 1) | (dfhog['V5_02'] == 1) | (dfhog['V5_03'] == 1)
dfhog.loc[mask_alguna_1, 'V5'] = 1

#print(dfhog['V5'].value_counts().sort_index())
dfhog.head()

,CODUSU,NRO_HOGAR,DECIFR,REGION,V5_01,V5_02,V5_03,V5
0,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2
1,TQSMNORTSHMOLTCDEGGFB00877605,1,6,42,2,2,2,2
2,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2
3,TQRMNOTTWHMMLOCDEFIAH00877819,1,12,43,2,2,2,2
4,TQRMNORQQHLMKOCDEHIBB00853810,2,1,40,2,2,2,2


In [5]:
#hogares: filtrar 50% más pobre (0 y 12)

dfhog_pobre = dfhog[dfhog['DECIFR'].isin([1, 2, 3, 4, 5])]
print(dfhog_pobre['DECIFR'].value_counts())

#persona: filtrar por Estado 1 (ocupado), 2 (desocupado) y 3 (inactivo)
dfusu = dfusu[dfusu['ESTADO'].isin([1, 2, 3])]
print(dfusu['ESTADO'].value_counts())

print("---------------------------------------------------")
dfhog_pobre.info()
dfusu.info()

DECIFR
1    1457
4    1350
2    1333
5    1287
3    1273
Name: count, dtype: int64
ESTADO
1    20076
3    18866
2     1381
Name: count, dtype: int64
---------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Index: 6700 entries, 0 to 15979
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CODUSU     6700 non-null   object
 1   NRO_HOGAR  6700 non-null   int64 
 2   DECIFR     6700 non-null   int64 
 3   REGION     6700 non-null   int64 
 4   V5_01      6700 non-null   int64 
 5   V5_02      6700 non-null   int64 
 6   V5_03      6700 non-null   int64 
 7   V5         6700 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 471.1+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 40323 entries, 0 to 45424
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      40323 non-null  object
 1   NRO_HOGAR   40323 non-null 

In [6]:
#Juntar tablas (left hogares pobres, right individuos)
df_join_pobre = dfhog_pobre.merge(dfusu, how='left', on=['CODUSU', 'NRO_HOGAR'])
df_join_pobre.info()
df_join_pobre.head(20)  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14030 entries, 0 to 14029
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      14030 non-null  object
 1   NRO_HOGAR   14030 non-null  int64 
 2   DECIFR      14030 non-null  int64 
 3   REGION      14030 non-null  int64 
 4   V5_01       14030 non-null  int64 
 5   V5_02       14030 non-null  int64 
 6   V5_03       14030 non-null  int64 
 7   V5          14030 non-null  int64 
 8   COMPONENTE  14030 non-null  int64 
 9   ESTADO      14030 non-null  int64 
 10  V5_01_M     14030 non-null  int64 
 11  V5_02_M     14030 non-null  int64 
 12  V5_03_M     14030 non-null  int64 
 13  PONDERA     14030 non-null  int64 
dtypes: int64(13), object(1)
memory usage: 1.5+ MB


,CODUSU,NRO_HOGAR,DECIFR,REGION,V5_01,V5_02,V5_03,V5,COMPONENTE,ESTADO,V5_01_M,V5_02_M,V5_03_M,PONDERA
0,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2,1,1,0,0,0,526
1,TQRMNOQVSHKOLNCDEGGFB00858441,1,1,42,2,2,2,2,2,3,0,0,0,526
2,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2,1,3,0,0,0,446
3,TQRMNOQQXHMMMLCDEGGFB00877606,1,4,42,2,2,2,2,2,3,0,0,0,446
4,TQRMNORQQHLMKOCDEHIBB00853810,2,1,40,2,2,2,2,1,3,0,0,0,132
5,TQRMNOTPYHMOLQCDEFMDB00877823,1,5,43,2,2,2,2,1,1,0,0,0,199
6,TQRMNOTPYHMOLQCDEFMDB00877823,1,5,43,2,2,2,2,2,1,0,0,0,199
7,TQRMNORVVHLLLPCDEFMDB00852190,1,3,43,2,2,2,2,1,3,0,0,0,146
8,TQRMNOQTTHKMKSCDEFMDB00857883,1,3,43,2,2,2,2,1,1,0,0,0,234
9,TQRMNOPXWHMOKSCDEOJAH00877825,1,3,44,2,2,2,2,1,1,0,0,0,113


In [7]:
# Crear UID único para el DataFrame completo
df_join_pobre['UID'] = (
    df_join_pobre['CODUSU'].astype(str) + '_' + 
    df_join_pobre['NRO_HOGAR'].astype(str) + '_' + 
    df_join_pobre['COMPONENTE'].astype(str)
)   
df_join_pobre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14030 entries, 0 to 14029
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      14030 non-null  object
 1   NRO_HOGAR   14030 non-null  int64 
 2   DECIFR      14030 non-null  int64 
 3   REGION      14030 non-null  int64 
 4   V5_01       14030 non-null  int64 
 5   V5_02       14030 non-null  int64 
 6   V5_03       14030 non-null  int64 
 7   V5          14030 non-null  int64 
 8   COMPONENTE  14030 non-null  int64 
 9   ESTADO      14030 non-null  int64 
 10  V5_01_M     14030 non-null  int64 
 11  V5_02_M     14030 non-null  int64 
 12  V5_03_M     14030 non-null  int64 
 13  PONDERA     14030 non-null  int64 
 14  UID         14030 non-null  object
dtypes: int64(13), object(2)
memory usage: 1.6+ MB


In [8]:
#chequeo previo a recodificar
print(df_join_pobre['ESTADO'].value_counts())
print(df_join_pobre['V5'].value_counts())
print(df_join_pobre['REGION'].value_counts())

# Recodificar variables (fusionar 1 y 2 como Activo; v5 con percibe/no-percibe)
df_join_pobre['ESTADO'] = df_join_pobre['ESTADO'].replace({1: 'Activo', 2: 'Activo', 3: 'Inactivo'})
df_join_pobre['V5'] = df_join_pobre['V5'].replace({1: 'Percibe', 2: 'No Percibe'})
df_join_pobre['REGION'] = df_join_pobre['REGION'].replace({1: 'Gran Buenos Aires', 40: 'Noroeste', 41: 'Noreste', 42: 'Cuyo', 43: 'Pampeana', 44: 'Patagonia'})

df_join_pobre.info()

ESTADO
3    7819
1    5547
2     664
Name: count, dtype: int64
V5
2    10492
1     3538
Name: count, dtype: int64
REGION
43    4106
40    3670
1     1751
41    1635
42    1577
44    1291
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14030 entries, 0 to 14029
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CODUSU      14030 non-null  object
 1   NRO_HOGAR   14030 non-null  int64 
 2   DECIFR      14030 non-null  int64 
 3   REGION      14030 non-null  object
 4   V5_01       14030 non-null  int64 
 5   V5_02       14030 non-null  int64 
 6   V5_03       14030 non-null  int64 
 7   V5          14030 non-null  object
 8   COMPONENTE  14030 non-null  int64 
 9   ESTADO      14030 non-null  object
 10  V5_01_M     14030 non-null  int64 
 11  V5_02_M     14030 non-null  int64 
 12  V5_03_M     14030 non-null  int64 
 13  PONDERA     14030 non-null  int64 
 14  UID         14030 non-null  objec

In [9]:
#ponderador casero
n = len(df_join_pobre)
N_t = df_join_pobre['PONDERA'].sum()
ajuste = n / N_t
df_join_pobre['PONDERA2'] = df_join_pobre['PONDERA'] * ajuste
#verif ponderadores
print(n)
print(df_join_pobre['PONDERA'].sum())
print(df_join_pobre['PONDERA2'].sum())

14030
8225518
14030.0


In [10]:
# Lista para guardar todas las tablas (Nacional + Regiones)
tablas_a_unir = []

# --- 1. TABLA NACIONAL (TOTAL) ---

# 1a. Calcular Absolutos Nacional
tabla_nac_abs = pd.pivot_table(
    df_join_pobre,
    values='PONDERA2',
    index='ESTADO',
    columns='V5',
    aggfunc='sum',
    margins=True,
    margins_name='Total',
    fill_value=0 # Rellenar con 0 si hay cruces vacíos
)

# 1b. Calcular Porcentajes Nacional
tabla_nac_pct = tabla_nac_abs.apply(lambda x: (x / x['Total']) * 100).round(2)

# 1c. Añadir el nivel 'Tipo' al índice
tabla_nac_abs['Tipo'] = 'Absoluto'
tabla_nac_pct['Tipo'] = 'Porcentaje'

# 1d. Apilar Abs y Pct
tabla_nacional_final = pd.concat([tabla_nac_abs, tabla_nac_pct])
tabla_nacional_final = tabla_nacional_final.set_index('Tipo', append=True)

# 1e. Añadir a la lista, etiquetando esta tabla como REGION = 'Total'
tablas_a_unir.append(
    pd.concat([tabla_nacional_final], keys=['Total (Nacional)'], names=['REGION'])
)


# --- 2. TABLAS PARCIALES (POR REGION) ---

regiones_ordenadas = sorted(df_join_pobre['REGION'].unique())

for region_id in regiones_ordenadas:
    df_region = df_join_pobre[df_join_pobre['REGION'] == region_id]
    
    if df_region.empty:
        continue
    
    # 2a. Calcular Absolutos para la Región
    tabla_reg_abs = pd.pivot_table(
        df_region,
        values='PONDERA2',
        index='ESTADO',
        columns='V5',
        aggfunc='sum',
        margins=True,
        margins_name='Total',
        fill_value=0
    )
    
    # 2b. Calcular Porcentajes para la Región
    tabla_reg_pct = tabla_reg_abs.apply(lambda x: (x / x['Total']) * 100).round(2)
    
    # 2c. Añadir el nivel 'Tipo'
    tabla_reg_abs['Tipo'] = 'Absoluto'
    tabla_reg_pct['Tipo'] = 'Porcentaje'
    
    # 2d. Apilar
    tabla_region_final = pd.concat([tabla_reg_abs, tabla_reg_pct])
    tabla_region_final = tabla_region_final.set_index('Tipo', append=True)
    
    # 2e. Añadir a la lista, etiquetando con el nombre de la Región
    tablas_a_unir.append(
        pd.concat([tabla_region_final], keys=[region_id], names=['REGION'])
    )




In [11]:

# --- Función de Ayuda para Chi-Cuadrado ---
# Esto crea un string con los resultados (Chi² y p-valor)
# Y añade un (*) si es significativo.
def get_chi2_results(abs_table):
    """Calcula Chi² y devuelve un string formateado."""
    
    # Quitar márgenes 'Total' para el cálculo
    table_no_totals = abs_table.iloc[:-1, :-1]
    
    if table_no_totals.empty:
        return "Datos insuficientes"
        
    try:
        chi2, p_valor, _, _ = chi2_contingency(table_no_totals)
        
        # Formatear el p-valor
        p_str = f"p={p_valor:.4f}"
        if p_valor < 0.05:
            p_str += " (*)" # Marcador de significancia
            
        return f"Chi²={chi2:.2f}; {p_str}"
        
    except ValueError as e:
        # Pasa si una fila/columna es todo 0
        return "Cálculo no válido"
# --- Fin de la Función de Ayuda ---


print("Generando reporte unificado con Chi-Cuadrado...")

# --- Preparación para el Excel ---
output_filename = 'analisis_completo_con_chi2.xlsx'
sheet_name = 'Analisis_Completo'

# Calcular dónde irá la columna de Chi²
# 3 niveles de índice (REGION, ESTADO, Tipo) 
index_cols_count = 3
# Columnas de datos (V5) + 1 (Total)
data_cols_count = df_join_pobre['V5'].nunique() + 1
# La columna de Chi² irá después de los índices y los datos + 1 columna de espacio
col_inicio_chi2 = index_cols_count + data_cols_count + 1

# Fila actual donde escribirá Excel
fila_actual = 0

with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
    
    # Escribir el título de la columna Chi²
    pd.DataFrame(['Análisis Chi² (χ²)']).to_excel(
        writer, sheet_name=sheet_name, 
        startrow=fila_actual, startcol=col_inicio_chi2, 
        header=False, index=False
    )
    fila_actual += 2 # Dejar espacio

    
    # --- 1. TABLA NACIONAL (TOTAL) ---
    
    # 1a. Calcular Absolutos Nacional
    tabla_nac_abs = pd.pivot_table(
        df_join_pobre, values='PONDERA2', index='ESTADO', columns='V5',
        aggfunc='sum', margins=True, margins_name='Total', fill_value=0
    )
    # 1b. Calcular Porcentajes Nacional
    tabla_nac_pct = tabla_nac_abs.apply(lambda x: (x / x['Total']) * 100).round(2)
    
    # 1c. Calcular Chi-Cuadrado Nacional
    chi2_nac_str = get_chi2_results(tabla_nac_abs)
    
    # 1d. Apilar Abs/Pct y añadir índice 'Tipo'
    tabla_nac_abs['Tipo'] = 'Absoluto'
    tabla_nac_pct['Tipo'] = 'Porcentaje'
    tabla_nacional_final = pd.concat([tabla_nac_abs, tabla_nac_pct]).set_index('Tipo', append=True)
    
    # 1e. Añadir índice 'REGION' y ordenar
    tabla_nacional_listo = pd.concat([tabla_nacional_final], keys=['Total (Nacional)'], names=['REGION'])
    tabla_nacional_listo = tabla_nacional_listo.sort_index()
    
    # 1f. Escribir la tabla principal a Excel
    tabla_nacional_listo.to_excel(
        writer, sheet_name=sheet_name, 
        startrow=fila_actual
    )
    
    # 1g. Escribir el resultado de Chi² alineado verticalmente
    fila_chi2 = fila_actual + (tabla_nacional_listo.shape[0] // 2) # Centrado
    pd.DataFrame([chi2_nac_str]).to_excel(
        writer, sheet_name=sheet_name, 
        startrow=fila_chi2, startcol=col_inicio_chi2, 
        header=False, index=False
    )
    
    # 1h. Actualizar la posición de la fila para la siguiente tabla
    fila_actual += tabla_nacional_listo.shape[0] + 2 # +2 para espacio

    
    # --- 2. TABLAS PARCIALES (POR REGION) ---
    
    regiones_ordenadas = sorted(df_join_pobre['REGION'].unique())
    
    for region_id in regiones_ordenadas:
        df_region = df_join_pobre[df_join_pobre['REGION'] == region_id]
        if df_region.empty:
            continue
        
        # 2a. Calcular Absolutos
        tabla_reg_abs = pd.pivot_table(
            df_region, values='PONDERA2', index='ESTADO', columns='V5',
            aggfunc='sum', margins=True, margins_name='Total', fill_value=0
        )
        # 2b. Calcular Porcentajes
        tabla_reg_pct = tabla_reg_abs.apply(lambda x: (x / x['Total']) * 100).round(2)
        
        # 2c. Calcular Chi-Cuadrado
        chi2_reg_str = get_chi2_results(tabla_reg_abs)
        
        # 2d. Apilar Abs/Pct y añadir índice 'Tipo'
        tabla_reg_abs['Tipo'] = 'Absoluto'
        tabla_reg_pct['Tipo'] = 'Porcentaje'
        tabla_region_final = pd.concat([tabla_reg_abs, tabla_reg_pct]).set_index('Tipo', append=True)
        
        # 2e. Añadir índice 'REGION' y ordenar
        tabla_region_listo = pd.concat([tabla_region_final], keys=[region_id], names=['REGION'])
        tabla_region_listo = tabla_region_listo.sort_index()
        
        # 2f. Escribir la tabla a Excel (sin encabezado esta vez)
        tabla_region_listo.to_excel(
            writer, sheet_name=sheet_name, 
            startrow=fila_actual, 
            header=False # El encabezado ya se escribió con la tabla nacional
        )
        
        # 2g. Escribir el resultado de Chi² alineado
        fila_chi2 = fila_actual + (tabla_region_listo.shape[0] // 2) # Centrado
        pd.DataFrame([chi2_reg_str]).to_excel(
            writer, sheet_name=sheet_name, 
            startrow=fila_chi2, startcol=col_inicio_chi2, 
            header=False, index=False
        )
        
        # 2h. Actualizar la fila
        fila_actual += tabla_region_listo.shape[0] + 2 # +2 para espacio

print(f"¡Listo! Análisis completo guardado en '{output_filename}'")

Generando reporte unificado con Chi-Cuadrado...
¡Listo! Análisis completo guardado en 'analisis_completo_con_chi2.xlsx'


In [14]:
# statsmodels necesita que la variable dependiente (Y) sea numérica (0 o 1).
# Vamos a crear 'ESTADO_BIN'. Asumiremos 'Activo' = 1 y 'Inactivo' = 0.
df_regresion = df_join_pobre.copy()
df_regresion['ESTADO_BIN'] = df_regresion['ESTADO'].apply(lambda x: 1 if x == 'Activo' else 0)

# La fórmula. statsmodels manejará 'V5' (Percibe/No Percibe) como una variable dummy.
# Automáticamente usará una categoría (alfabéticamente, 'No Percibe') como base.
formula = 'ESTADO_BIN ~ V5'

# --- PASO 2: Función de Ayuda para R² de Nagelkerke ---
# statsmodels por defecto da el R² de McFadden, así que calculamos el de Nagelkerke.
def get_nagelkerke_r2(results):
    """Calcula el Pseudo R² de Nagelkerke a partir de un resultado de statsmodels."""
    try:
        llf = results.llf  # Log-likelihood del modelo completo
        lln = results.llnull # Log-likelihood del modelo nulo (solo intercepto)
        n = results.nobs   # Número de observaciones (suma de ponderadores)
        
        # Calcular R² de Cox & Snell
        r2_cs = 1 - (np.exp(lln - llf))**(2/n)
        
        # Calcular el R² máximo
        r2_max_nagel = 1 - (np.exp(lln))**(2/n)
        
        if r2_max_nagel == 0:
            return 0.0
            
        return r2_cs / r2_max_nagel
    except Exception:
        return np.nan

# --- PASO 3: Correr modelos y guardar resultados ---
results_list = []

# Corremos primero el modelo Nacional
print("Calculando Regresión Logística Nacional...")
try:
    df_nat = df_regresion
    pesos_nat = df_nat['PONDERA2']
    
    # Usamos GLM (Modelo Lineal Generalizado) con familia Binomial (logística)
    # y 'freq_weights' para aplicar el ponderador PONDERA2.
    modelo_nat = smf.glm(formula=formula, 
                         data=df_nat, 
                         family=sm.families.Binomial(), 
                         freq_weights=pesos_nat)
    results_nat = modelo_nat.fit()
    
    # Extraer resultados
    coefs_nat = results_nat.params
    pvals_nat = results_nat.pvalues
    nagel_nat = get_nagelkerke_r2(results_nat)
    
    # Detectar el nombre de la columna V5 (ej. "V5[T.Percibe]")
    v5_col_name = [col for col in coefs_nat.index if 'V5[' in col][0]
    
    results_list.append({
        'Modelo': 'Total (Nacional)',
        'Beta (Intercept)': coefs_nat['Intercept'],
        f'Beta ({v5_col_name})': coefs_nat[v5_col_name],
        'P-Valor (Intercept)': pvals_nat['Intercept'],
        f'P-Valor ({v5_col_name})': pvals_nat[v5_col_name],
        'Nagelkerke R²': nagel_nat,
        'N (Suma Ponderador)': results_nat.nobs
    })
    
except Exception as e:
    print(f"Error al calcular modelo nacional: {e}")

# Ahora iteramos y corremos el modelo para CADA REGIÓN
print("Calculando Regresiones Logísticas por Región...")
regiones_ordenadas = sorted(df_regresion['REGION'].unique())

for region_id in regiones_ordenadas:
    df_region = df_regresion[df_regresion['REGION'] == region_id]
    
    # Asegurarnos de que hay suficientes datos y variabilidad para correr el modelo
    if df_region.empty or df_region['V5'].nunique() < 2 or df_region['ESTADO_BIN'].nunique() < 2:
        print(f"  > Omitiendo Región {region_id}: datos insuficientes o sin variabilidad.")
        continue
        
    try:
        pesos_reg = df_region['PONDERA2']
        modelo_reg = smf.glm(formula=formula, 
                             data=df_region, 
                             family=sm.families.Binomial(), 
                             freq_weights=pesos_reg)
        results_reg = modelo_reg.fit()
        
        # Extraer resultados
        coefs_reg = results_reg.params
        pvals_reg = results_reg.pvalues
        nagel_reg = get_nagelkerke_r2(results_reg)
        
        v5_col_name_reg = [col for col in coefs_reg.index if 'V5[' in col][0]
        
        results_list.append({
            'Modelo': region_id,
            'Beta (Intercept)': coefs_reg['Intercept'],
            f'Beta ({v5_col_name})': coefs_reg[v5_col_name_reg],
            'P-Valor (Intercept)': pvals_reg['Intercept'],
            f'P-Valor ({v5_col_name})': pvals_reg[v5_col_name_reg],
            'Nagelkerke R²': nagel_reg,
            'N (Suma Ponderador)': results_reg.nobs
        })
        print(f"  > Región {region_id} calculada.")
        
    except Exception as e:
        print(f"  > Error al calcular modelo para {region_id}: {e}")

# --- PASO 4: Presentar los resultados en una tabla ---
if results_list:
    df_summary = pd.DataFrame(results_list)
    df_summary = df_summary.set_index('Modelo')
    
    print("\n" + "="*50)
    print("RESULTADOS DE LA REGRESIÓN LOGÍSTICA")
    print("="*50)
    # Usamos .to_string() para asegurarnos de que se impriman todas las columnas
    print(df_summary.to_string())
else:
    print("No se pudieron generar resultados de regresión.")

print("\n--- Fin del Análisis de Regresión ---")

Calculando Regresión Logística Nacional...
Calculando Regresiones Logísticas por Región...
  > Región Cuyo calculada.
  > Región Gran Buenos Aires calculada.
  > Región Noreste calculada.
  > Región Noroeste calculada.
  > Región Pampeana calculada.
  > Región Patagonia calculada.

RESULTADOS DE LA REGRESIÓN LOGÍSTICA
                   Beta (Intercept)  Beta (V5[T.Percibe])  P-Valor (Intercept)  P-Valor (V5[T.Percibe])  Nagelkerke R²  N (Suma Ponderador)
Modelo                                                                                                                                     
Total (Nacional)          -0.207465              0.074837         7.590749e-25                 0.044830       0.000287                14030
Cuyo                      -0.122802             -0.010573         9.480476e-02                 0.941435       0.000006                 1577
Gran Buenos Aires         -0.222493              0.174601         2.757961e-14                 0.001145       0.006019  